<a href="https://colab.research.google.com/github/nadyadtm/Document-Retrieval/blob/main/Information_Retrieval_Evaluation_BERT_TFIDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Evaluasi dari Nadya Aditama (23520039)

# Information Retrieval Evaluation

Berikut ini adalah evaluasi dari information retrieval mengenai data COVID 19 dengan menggunakan Sentence BERT dan TF-IDF

## Load Data dan Import Package

In [1]:
# Menginstall Sentence-Transformers
!pip install sentence-transformers

     |████████████████████████████████| 81kB 9.4MB/s 
     |████████████████████████████████| 2.1MB 28.0MB/s 
     |████████████████████████████████| 1.2MB 53.7MB/s 
     |████████████████████████████████| 870kB 54.1MB/s 
     |████████████████████████████████| 3.3MB 46.7MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-1.0.4-cp37-none-any.whl size=114307 sha256=ad0401abe852ee380b53bc1f36672524b8ab9b6e39836e17270ea027e26e723a
  Stored in directory: /root/.cache/pip/wheels/26/ea/89/d0d2e013d951b6d23270aa9ca4018b82632ab7cd933c331316
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=ea7747f9986ddcc3b0b9a5ba1f423bfbb9520b8d297617cddaf898ba502ad953
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sentence-transformers sacremoses


In [2]:
!wget 'https://raw.githubusercontent.com/deepset-ai/COVID-QA/master/data/question-answering/COVID-QA.json'

--2021-04-18 04:28:07--  https://raw.githubusercontent.com/deepset-ai/COVID-QA/master/data/question-answering/COVID-QA.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4418117 (4.2M) [text/plain]
Saving to: ‘COVID-QA.json’

COVID-QA.json       100%[===================>]   4.21M  --.-KB/s    in 0.05s   

2021-04-18 04:28:07 (90.1 MB/s) - ‘COVID-QA.json’ saved [4418117/4418117]



In [3]:
raw_file = './COVID-QA.json'

In [4]:
import pandas as pd
import numpy as np
data = pd.read_json(raw_file)
data = pd.json_normalize(data['data'], record_path='paragraphs')
new_data = pd.DataFrame(0, index=np.arange(2019),columns=["document_id", "context", "id_question", "question", "answers_start", "answers_text", "is_impossible"])
idx = 0
for x in range(0,len(data['qas'])):
  for y in range(0, len(data['qas'][x])):
    new_data['document_id'].iloc[idx]=data['document_id'][x]
    new_data['context'].iloc[idx]=data['context'][x]
    new_data['id_question'].iloc[idx] = data['qas'][x][y]['id']
    new_data['question'].iloc[idx] = data['qas'][x][y]['question']
    new_data['answers_start'].iloc[idx] = data['qas'][x][y]['answers'][0]['answer_start']
    new_data['answers_text'].iloc[idx] = data['qas'][x][y]['answers'][0]['text']
    new_data['is_impossible'].iloc[idx] = data['qas'][x][y]['is_impossible']
    idx+=1
new_data


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


,document_id,context,id_question,question,answers_start,answers_text,is_impossible
0,630,Functional Genetic Variants in DC-SIGNR Are As...,262,What is the main cause of HIV-1 infection in c...,370,Mother-to-child transmission (MTCT) is the mai...,False
1,630,Functional Genetic Variants in DC-SIGNR Are As...,276,What plays the crucial role in the Mother to C...,2003,DC-SIGNR plays a crucial role in MTCT of HIV-1...,False
2,630,Functional Genetic Variants in DC-SIGNR Are As...,278,How many children were infected by HIV-1 in 20...,2291,"more than 400,000 children were infected world...",False
3,630,Functional Genetic Variants in DC-SIGNR Are As...,316,What is the role of C-C Motif Chemokine Ligand...,28143,"High copy numbers of CCL3L1, a potent HIV-1 su...",False
4,630,Functional Genetic Variants in DC-SIGNR Are As...,305,What is DC-GENR and where is it expressed?,3207,Dendritic cell-specific ICAM-grabbing non-inte...,False
...,...,...,...,...,...,...,...
2014,1713,"Ebola Virus Maintenance: If Not (Only) Bats, W...",5315,What is the structure of the Ebolavirus?,2270,single-strand RNA filoviruses,False
2015,1713,"Ebola Virus Maintenance: If Not (Only) Bats, W...",5316,When was the West African Ebolavirus outbreak?,2546,2013-2016,False
2016,1713,"Ebola Virus Maintenance: If Not (Only) Bats, W...",5317,What animals are considered to be maintenance ...,4083,African bats,False
2017,1713,"Ebola Virus Maintenance: If Not (Only) Bats, W...",5318,What do circles indicate in Figure 1?,7212,a maintenance function play by the host(s),False


## Mengambil Dokumen yang Tersedia
Pada bagian ini, dilakukan pengambilan seluruh dokumen pada dataset

In [5]:
#ambil context dan question
docs_train = new_data[['document_id','context']]

In [6]:
docs_train.drop_duplicates(subset=None, keep='first', inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


## Preprocessing
Pada bagian ini dilakukan preprocessing dengan menghilangkan sitasi, link, dan digit, kemudian melakukan lower case, stopword removal, dan lemmatisasi

In [7]:
#package
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('stopwords')

nltk.download('averaged_perceptron_tagger')
from nltk import pos_tag

nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
import re
def punc_remove(sentence):
  punc = re.compile("[^\w\s]")
  # Regex to check valid URL
  regex_link = ("((http|https)://)(www.)?" +
             "[a-zA-Z0-9@:%._\\+~#?&//=]" +
             "{2,256}\\.[a-z]" +
             "{2,6}\\b([-a-zA-Z0-9@:%" +
             "._\\+~#?&//=]*)")
  citation = ("(\[\d+-\d+\]|\[\d+(, \d+)*\])")
     
  # Compile the ReGex
  p = re.compile(regex_link)
  r = re.compile(citation)
  digits = re.compile('[0-9]')


  sentence = sentence.replace('\n',' ')
  sentence = p.sub(' ',sentence)
  sentence = r.sub(' ',sentence)
  test_str = digits.sub(' ',sentence)
  test_str = test_str.split(' ')

  x = ' '.join(test_str).split()
  y = ' '.join(x)

  return y
  
def clean_data(text):
  lem = WordNetLemmatizer()
  lower_case = text.lower()
  tokenized = word_tokenize(lower_case)
  data_lemmatize=[lem.lemmatize(word) for word in tokenized if not word in stopwords.words('english')]
  data_fixed = ' '.join(data_lemmatize)
  return data_fixed

In [9]:
docs_clean_train = [clean_data(punc_remove(sentence)) for sentence in docs_train['context']]

In [10]:
docs_train['cleaned_data'] = docs_clean_train

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


## TF IDF Feature
Pada modul ini, dilakukan pengambilan fitur TF-IDF dengan max_features 2000

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer

In [12]:
tfidfvectorizer = TfidfVectorizer(analyzer='word',stop_words= 'english', max_features=2000)
fixtrain = tfidfvectorizer.fit_transform(docs_train['cleaned_data']).toarray()

In [13]:
import pandas as pd
df_tfidf = pd.DataFrame(fixtrain, columns=tfidfvectorizer.get_feature_names())
df_tfidf.head(5)

,aa,ab,ability,able,absence,abstract,abundance,access,accession,accessory,accordance,according,account,accumulation,accuracy,accurate,ace,achieved,acid,acquired,acquisition,act,action,activate,activated,activation,active,activity,acute,ad,adapted,adaptive,added,addition,additional,additionally,address,adenovirus,adg,adhesion,...,washing,water,wave,way,week,weekly,weight,west,western,white,wide,widely,widespread,wild,winter,woman,work,worker,working,world,worldwide,wt,wuhan,www,year,yfp,yield,yn,young,zanamivir,zhang,zinc,zm,zone,zoonotic,µl,µm,μg,μl,μm
0,0.019085,0.000000,0.000000,0.00000,0.000000,0.001578,0.005157,0.003987,0.000000,0.0,0.0,0.018534,0.000000,0.0,0.0,0.0,0.0,0.0,0.008691,0.0,0.0,0.003584,0.0,0.0,0.0,0.0,0.0,0.020155,0.00000,0.0,0.004207,0.004463,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.005528,...,0.0,0.0,0.0,0.002897,0.014237,0.0,0.0,0.0,0.004093,0.0,0.0,0.0,0.0,0.017535,0.000000,0.004267,0.0,0.000000,0.0,0.002621,0.006728,0.078216,0.0,0.004039,0.002300,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0000,0.000000
1,0.000000,0.000000,0.001572,0.00296,0.027181,0.000841,0.000000,0.000000,0.000000,0.0,0.0,0.001235,0.003953,0.0,0.0,0.0,0.0,0.0,0.026258,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.024179,0.00000,0.0,0.000000,0.000000,0.004716,0.011762,0.0,0.000000,0.002182,0.0,0.0,0.002947,...,0.0,0.0,0.0,0.001545,0.000000,0.0,0.0,0.0,0.026186,0.0,0.0,0.0,0.0,0.014959,0.000000,0.000000,0.0,0.004198,0.0,0.000000,0.000000,0.048119,0.0,0.002153,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.004999,0.0025,0.007921
2,0.000000,0.216544,0.000000,0.00000,0.000000,0.016696,0.000000,0.000000,0.283367,0.0,0.0,0.024516,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.030130,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.048777,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0000,0.000000
3,0.000000,0.000000,0.000000,0.00000,0.000000,0.008809,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.02587,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.019374,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.023823,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.025679,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0000,0.000000
4,0.000000,0.000000,0.000000,0.00000,0.000000,0.006085,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.044690,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.007734,0.0,0.000000,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.010981,0.0,0.0,0.0,0.015784,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.008869,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0000,0.000000


In [14]:
docs_train['vector_tf_idf']=df_tfidf.values.tolist()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [15]:
docs_train.head()

,document_id,context,cleaned_data,vector_tf_idf
0,630,Functional Genetic Variants in DC-SIGNR Are As...,functional genetic variant dc-signr associated...,"[0.019085058375148987, 0.0, 0.0, 0.0, 0.0, 0.0..."
11,650,Role of S-Palmitoylation on IFITM5 for the Int...,role s-palmitoylation ifitm interaction fkbp o...,"[0.0, 0.0, 0.0015718760321186836, 0.0029601433..."
22,1546,First Complete Genome Sequence of a French Bov...,first complete genome sequence french bovine c...,"[0.0, 0.21654350686727633, 0.0, 0.0, 0.0, 0.01..."
27,1545,Species‐specific clinical characteristics of h...,species‐specific clinical characteristic human...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.008809309160605168..."
34,1552,One step closer to an experimental infection s...,one step closer experimental infection system ...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.006085110520319744..."


## Sentence Bert Model
Pada modul ini, dilakukan embedding dengan model Sentence BERT dengan arsitektur pre-trained distilbert-base-nli-mean-tokens

In [16]:
from sentence_transformers import SentenceTransformer

Training Sentence-BERT dengan database dokumen

In [17]:
import torch

if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: Tesla T4


In [18]:
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader

# Mendefinisikan Model
model = SentenceTransformer('distilbert-base-nli-mean-tokens')
# Mendefinisikan data train
train_examples = [InputExample(texts=docs_train['cleaned_data'].to_list(), label=0.8)]

#Mendefinisikan the dataloader and the train loss
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
train_loss = losses.CosineSimilarityLoss(model)

#Fit Model
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=3, warmup_steps=100)

Melakukan embedding dengan model yang sudah di train sebelumnya

In [19]:
def sentence_embeddings(docs):
  sentence_embeddings = model.encode(docs)
  return sentence_embeddings

In [20]:
x=sentence_embeddings(docs_train['cleaned_data'].to_list())

In [21]:
x[0].shape

(768,)

In [22]:
y=pd.DataFrame(x)
docs_train['vector_bert']=y.values.tolist()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


## Similarity
Kemudian dibuat fungsi similarity untuk kedua kasus, yaitu ranking_ir_bert() dan ranking_ir_tf_idf(). Kemudian akan diambil 10 dokumen terbaik berdasarkan similarity terbesar

In [23]:
from sklearn.metrics.pairwise import cosine_similarity

Fungsi similarity ranking bert

In [24]:
def ranking_ir_bert(query):

  query = clean_data(punc_remove(query))

  # mengenerate vektor
  vector=sentence_embeddings([query])
  
  # perankingan dokumen dengan cosine similarity
  documents=docs_train.copy()
  documents['similarity']=documents['vector_bert'].apply(lambda x: cosine_similarity(np.array(vector).reshape(1, -1),np.array(x).reshape(1, -1)).item())
  documents.sort_values(by='similarity',ascending=False,inplace=True)
  
  return documents[['document_id','context','similarity']].head(10).reset_index(drop=True)

Fungsi similarity ranking tf idf

In [25]:
def ranking_ir_tf_idf(query):

  query = clean_data(punc_remove(query))

  # mengenerate vektor
  vector=tfidfvectorizer.transform([query]).toarray()
  
  # perankingan dokumen dengan cosine similarity
  documents=docs_train.copy()
  documents['similarity']=documents['vector_tf_idf'].apply(lambda x: cosine_similarity(np.array(vector).reshape(1, -1),np.array(x).reshape(1, -1)).item())
  documents.sort_values(by='similarity',ascending=False,inplace=True)
  
  return documents[['document_id','context','similarity']].head(10).reset_index(drop=True)

### Percobaan dengan contoh input keyword

In [26]:
ranking_ir_bert('main cause COVID-19')

,document_id,context,similarity
0,650,Role of S-Palmitoylation on IFITM5 for the Int...,0.429264
1,1560,Relationship between hepcidin and oxidant/anti...,0.403205
2,1586,In Vitro Bactericidal Activity of 4- and 5-Chl...,0.364596
3,1565,"Design, Synthesis, Evaluation and Thermodynami...",0.364080
4,1561,Acute Hemorrhagic Encephalitis Responding to C...,0.358630
5,2450,Safe patient transport for COVID-19\n\nhttps:/...,0.354372
6,1620,A missense mutation in Katnal1 underlies behav...,0.349286
7,1621,Vesicular stomatitis virus with the rabies vir...,0.348833
8,1686,Nucleolar Protein Trafficking in Response to H...,0.340065
9,1667,C. difficile 630Δerm Spo0A Regulates Sporulati...,0.332736


In [27]:
ranking_ir_tf_idf('main cause COVID-19')

,document_id,context,similarity
0,185,"CDC Summary 21 MAR 2020,\nhttps://www.cdc.gov/...",0.557419
1,2450,Safe patient transport for COVID-19\n\nhttps:/...,0.363624
2,188,The Battle Against Coronavirus Disease 2019 (C...,0.317647
3,1559,COVID-19 and smoking: A systematic review of t...,0.248786
4,2522,Identification of COVID-19 Can be Quicker thro...,0.234826
5,2432,Factors Associated With Mental Health Outcomes...,0.194040
6,2642,First cases of coronavirus disease 2019 (COVID...,0.159228
7,2643,Responding to the COVID-19 pandemic in complex...,0.147440
8,2440,Optimization Method for Forecasting Confirmed ...,0.135337
9,2683,Estimating the number of infections and the im...,0.129470


## Pengujian dengan Keyword

### Fungsi Keyword 
Melakukan pengambilan keyword seluruh question dalam modul Question Understanding

Fungsi ekstraksi keyword

In [28]:
def extract_keyword(postag):
  keyword = ""
  for i in range(0,len(postag)):
    if postag[i][1]=="NNP" or postag[i][1]=="NNS" or postag[i][1]=="NN" or (postag[i][1]=="JJ" and postag[i][0].lower()!="many") or postag[i][1]=="CD" or postag[i][1]=="RBS" or (postag[i][1]=="VBN" and postag[i][1]!="been") or (postag[i][1]=="VBD" and postag[i][0].lower()!="was" and postag[i][0].lower()!="were") or postag[i][1]=="VBG" or (postag[i][1]=="VB" and postag[i][0].lower()!="be") or postag[i][1]=="RB":
      keyword += postag[i][0] +" "
  if len(keyword)!=0 and keyword[len(keyword)-1] == " ":
    keyword = keyword[:-1]
  return keyword

Fungsi ekstraksi question type

In [29]:
def question_type_extraction(question):
  qarray = [0,0,0,0,0,0,0,0,0]
  labq = ["What","Where","Why","Who","When","How much","How many","Which","How"]
  qtype = ""
  what = 0
  where = 1
  why = 2
  who = 3
  when = 4
  howmuch = 5
  howmany = 6
  which = 7
  how = 8

  for i in range(0,len(question)):
    if question[i].lower() == "what":
      qarray[what] = 1
    elif question[i].lower() == "where":
      qarray[where] = 1
    elif question[i].lower() == "why":
      qarray[why] = 1
    elif question[i].lower() == "who":
      qarray[who] = 1
    elif question[i].lower() == "when":
      qarray[when] = 1
    elif question[i].lower() == "which":
      qarray[which] = 1
    elif question[i].lower() == "how":
      if question[i+1].lower() != "much" and question[i+1].lower() != "many":
        qarray[how] = 1
      elif question[i+1].lower() == "much":
        qarray[howmuch] = 1
      elif question[i+1].lower() == "many":
        qarray[howmany] = 1

  
  if 1 in qarray:
    for i in range(0,len(qarray)):
      if qarray[i]==1:
        qtype += labq[i] + ","
  else:
    qtype="Unknown"

  if qtype[len(qtype)-1] == ",":
    qtype = qtype[:-1]
  return qtype

Fungsi question understanding

In [30]:
 def question_understanding(dataset):
  keywords = []
  qtype = []
  answer = []

  for i in range(0,len(dataset)):
    sentence = pos_tag(word_tokenize(dataset.question[i]))
    keywords.append(extract_keyword(sentence))
    qtype.append(question_type_extraction(word_tokenize(dataset.question[i])))
    ans = dataset.answers_text[i]
    answer.append(ans)
  
  output = pd.DataFrame({"keywords":np.array(keywords),
                         "question_type":np.array(qtype),
                         "answer":np.array(answer)})

  return output

### Evaluasi

#### TF IDF

In [31]:
qu = question_understanding(new_data)
qu['docs_id']=new_data['document_id']

In [32]:
qu['docs_recc']=qu['keywords'].apply(lambda x : ranking_ir_tf_idf(x))

Pada evaluasi ini, dilakukan pengecekan pada masing-masing apakah dokumen dari keyword tersebut termasuk ke dalam 10 dokumen terbaik yang didapat dari keyword

In [33]:
is_docs_there=[]
for idx,item in qu.iterrows():
  is_docs_there.append(item['docs_id'] in qu['docs_recc'][idx]['document_id'].unique())

In [34]:
qu['is_found']=is_docs_there
print(qu['is_found'].sum()/2019)

0.6988608221892025


Berdasarkan hasil evaluasi, dari 2019 pertanyaan, 69% pertanyaan berhasil mendapatkan dokumen yang sesuai berdasarkan 10 dokumen yang diambil sebelumnya

#### BERT

In [35]:
qu = question_understanding(new_data)
qu['docs_id']=new_data['document_id']

In [36]:
qu['docs_recc']=qu['keywords'].apply(lambda x : ranking_ir_bert(x))

Pada evaluasi ini, dilakukan pengecekan pada masing-masing apakah dokumen dari keyword tersebut termasuk ke dalam 10 dokumen terbaik yang didapat dari keyword

In [37]:
is_docs_there=[]
for idx,item in qu.iterrows():
  is_docs_there.append(item['docs_id'] in qu['docs_recc'][idx]['document_id'].unique())

In [38]:
qu['is_found']=is_docs_there
print(qu['is_found'].sum()/2019)

0.23476968796433878


Berdasarkan hasil evaluasi, dari 2019 pertanyaan, 23.4% pertanyaan berhasil mendapatkan dokumen yang sesuai berdasarkan 10 dokumen yang diambil sebelumnya

#### Keyword yang tidak jelas

Pada dataset terdapat keyword yang tidak jelas seperti pada cell dibawah ini, sehingga dokumen yang di retrieve tidak benar

In [39]:
qu.tail(1)

,keywords,question_type,answer,docs_id,docs_recc,is_found
2018,arrows indicate Figure 1,What,infectious transmission pathways between hosts,1713,document_id ...,False


In [40]:
ranking_ir_tf_idf(qu.keywords[2018])

,document_id,context,similarity
0,2683,Estimating the number of infections and the im...,0.064803
1,2519,Detectable 2019-nCoV viral RNA in blood is a s...,0.064411
2,186,Identifying Locations with Possible Undetected...,0.061561
3,1621,Vesicular stomatitis virus with the rabies vir...,0.051927
4,1576,Characterization of a New Member of Alphacoron...,0.051283
5,1698,"Accelerated viral dynamics in bat cell lines, ...",0.049541
6,1662,The Evolutionary Dynamics of the Lion Panthera...,0.049041
7,1596,Glycyrrhizin Exerts Antioxidative Effects in H...,0.047571
8,1620,A missense mutation in Katnal1 underlies behav...,0.047453
9,1585,Immunomodulatory Activity and Protective Effec...,0.042563
